In [1]:
# Old method
from pkg_resources import working_set
from src.license_scanner.parse_license import parse_license

unknown_license = "UNKNOWN"
all_packages_old = []


for key in working_set.normalized_to_canonical_keys:
    """
    The metadata of a package is stored in "metadata" or "PKG-INFO"
    """
    try:
        package_name = working_set.normalized_to_canonical_keys[key]
        generic_name = package_name.replace("-", "_").lower()
        package = working_set.by_key[package_name]
        metadata_lines = package.get_metadata("METADATA").split("\n")
    except:
        try:
            metadata_lines = package.get_metadata("PKG-INFO").split("\n")
        except:
            all_packages_old.append([package_name, unknown_license, generic_name])
            continue

    license_arg_raw = ""
    license_classifier_raw = ""
    for line in metadata_lines:
        line: str = line
        if line.startswith("License-Expression: "):
            license_arg_raw = line.replace("License-Expression: ", "")
        if line.startswith("License: "):
            license_arg_raw = line.replace("License: ", "")
        if line.startswith("Classifier: License ::"):
            license_classifier_raw = line.split(" :: ")[-1]
            if license_classifier_raw.lower() in ["osi approved"]:
                license_classifier_raw = ""

    general_license = "NOT FOUND"
    license_arg = parse_license(license_arg_raw)
    license_classifier = parse_license(license_classifier_raw)

    # You can get the license from license argument or classifier
    if license_arg is not "" and license_arg != unknown_license:
        general_license = license_arg
    elif license_classifier:
        general_license = license_classifier
    else:
        general_license = unknown_license

    all_packages_old.append([package_name, general_license, generic_name])

all_packages_old.sort()
print("Number of packages found (old method):", len(all_packages_old))
all_packages_old

<>:43: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
/tmp/ipykernel_47788/4015494294.py:2: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import working_set


__phello__
_pyrepl
asyncio
collections
concurrent
ctypes
curses
dbm
email
encodings
ensurepip
html
http
idlelib
importlib
json
logging
multiprocessing
pathlib
pydoc_data
re
sqlite3
sysconfig
test
tkinter
tomllib
turtledemo
unittest
urllib
venv
wsgiref
xml
xmlrpc
zipfile
zoneinfo
Crypto
Cryptodome
Cython
DateTime
IPython
Levenshtein
OpenSSL
PIL
PyPDF2
TCLIService
_argon2_cffi_bindings
_distutils_hack
_multiprocess
_plotly_utils
_pytest
_soundfile_data
_yaml
absl
accelerate
acme
adal
adlfs
aenum
agate
aioboto3
aiobotocore
aiodns
aiofiles
aiohappyeyeballs
aiohttp
aiohttp_cors
aiohttp_retry
aioitertools
aioresponses
aiosignal
aiosqlite
alabaster
alembic
aliyunsdkcore
altair
altgraph
amqp
aniso8601
annotated_types
ansible
ansible_test
anthropic
antlr4
anyio
apache_beam
apiclient
apispec
applicationinsights
appnope
apscheduler
argcomplete
argon2
arrow
asgiref
asn1crypto
astor
astroid
asttokens
astunparse
async_lru
async_timeout
asyncpg
asyncssh
asynctest
atlassian
attr
attrs
authlib
autocomm

/tmp/ipykernel_47788/4015494294.py:43: SyntaxWarning: "is not" with 'str' literal. Did you mean "!="?
  if license_arg is not "" and license_arg != unknown_license:


[['absl-py', 'Apache license 2.0', 'absl_py'],
 ['accelerate', 'Apache license', 'accelerate'],
 ['acme', 'Apache license 2.0', 'acme'],
 ['adal', 'MIT license', 'adal'],
 ['adlfs', 'BSD license', 'adlfs'],
 ['aenum', 'BSD license', 'aenum'],
 ['agate', 'MIT license', 'agate'],
 ['aioboto3', 'Apache license 2.0', 'aioboto3'],
 ['aiobotocore', 'Apache license 2.0', 'aiobotocore'],
 ['aiodns', 'MIT license', 'aiodns'],
 ['aiofiles', 'Apache license 2.0', 'aiofiles'],
 ['aiohappyeyeballs',
  'Python software foundation license v2',
  'aiohappyeyeballs'],
 ['aiohttp', None, 'aiohttp'],
 ['aiohttp-cors', 'Apache license 2.0', 'aiohttp_cors'],
 ['aiohttp-retry', 'MIT license', 'aiohttp_retry'],
 ['aioitertools', None, 'aioitertools'],
 ['aioresponses', None, 'aioresponses'],
 ['aiosignal', 'Apache license 2.0', 'aiosignal'],
 ['aiosqlite', None, 'aiosqlite'],
 ['alabaster', None, 'alabaster'],
 ['alembic', 'MIT license', 'alembic'],
 ['aliyun-python-sdk-core', 'Apache license 2.0', 'aliyun_p

In [2]:
from copy_pkg_resources_2 import get_all_package_names

all_packages = get_all_package_names()
print("Number of packages found (new method):", len(all_packages))

Number of packages found (new method): 1162


In [3]:
import importlib_metadata as im


def get_license(name: str) -> str:
    try:
        metas = im.metadata(name)
    except im.PackageNotFoundError:
        return unknown_license
    all_metas = str(metas).split("\n")
    for m in all_metas:
        if m.startswith("License-Expression: "):
            license = m.replace("License-Expression: ", "")
            if license and license != "UNKNOWN":
                return parse_license(license)
        if m.startswith("License: "):
            license = m.replace("License: ", "")
            if license and license != "UNKNOWN":
                return parse_license(license)
        if m.startswith("Classifier: License ::"):
            license = m.split("::")[-1]
            if license and license != "UNKNOWN":
                return parse_license(license)

    return unknown_license

In [4]:
all_packages_new = []
for name in get_all_package_names():
    try:
        generic_name = name.replace("-", "_").lower()
        all_packages_new.append([name, get_license(name), generic_name])
    except:
        print(f"Failed to get license for {name}")
        continue

all_packages_new.sort()
all_packages_new

[['APScheduler', 'MIT license', 'apscheduler'],
 ['Brotli', 'MIT license', 'brotli'],
 ['DateTime', 'Zope Public License 2.1', 'datetime'],
 ['Deprecated', 'MIT license', 'deprecated'],
 ['Events', 'BSD license', 'events'],
 ['Flask-AppBuilder', 'BSD license', 'flask_appbuilder'],
 ['Flask-JWT-Extended', 'MIT license', 'flask_jwt_extended'],
 ['Flask-Login', 'MIT license', 'flask_login'],
 ['JayDeBeApi', None, 'jaydebeapi'],
 ['Office365-REST-Python-Client',
  'MIT license',
  'office365_rest_python_client'],
 ['PyJWT', 'MIT license', 'pyjwt'],
 ['PySocks', 'BSD license', 'pysocks'],
 ['SQLAlchemy-JSONField', 'Apache license 2.0', 'sqlalchemy_jsonfield'],
 ['Send2Trash', 'BSD license', 'send2trash'],
 ['StrEnum', 'MIT license', 'strenum'],
 ['Unidecode', 'GNU general public license', 'unidecode'],
 ['absl-py', 'Apache license 2.0', 'absl_py'],
 ['accelerate', 'Apache license', 'accelerate'],
 ['acme', 'Apache license 2.0', 'acme'],
 ['adal', 'MIT license', 'adal'],
 ['adlfs', 'BSD lice

In [5]:
import pandas as pd

all_packages = pd.DataFrame(
    columns=[
        "package_name_old",
        "package_name_new",
        "package_name_general",
        "license_new",
        "license_old",
    ]
)

all_packages_new_2 = all_packages_new.copy()

# All old packages
for i in all_packages_old:
    new_package = []
    # Match with new packages
    for j in all_packages_new_2:
        if j[2] == i[2]:
            new_package = j
            all_packages_new_2.remove(j)

    new_row = pd.Series(
        {
            "package_name_old": i[0].strip(),
            "package_name_new": new_package[0].strip() if new_package else "",
            "package_name_general": i[2].strip(),
            "license_new": new_package[1].strip() if new_package[1] else "",
            "license_old": i[1].strip() if i[1] else "",
        }
    )
    all_packages = pd.concat([all_packages, new_row.to_frame().T])

# Add new packages NOT matches an old one
for i in all_packages_new_2:

    new_row = pd.Series(
        {
            "package_name_new": i[0].strip(),
            "package_name_general": i[2].strip(),
            "license_new": i[1].strip(),
        }
    )
    all_packages = pd.concat([all_packages, new_row.to_frame().T])

all_packages = all_packages.fillna("")
all_packages.head()

,package_name_old,package_name_new,package_name_general,license_new,license_old
0,absl-py,absl-py,absl_py,Apache license 2.0,Apache license 2.0
0,accelerate,accelerate,accelerate,Apache license,Apache license
0,acme,acme,acme,Apache license 2.0,Apache license 2.0
0,adal,adal,adal,MIT license,MIT license
0,adlfs,adlfs,adlfs,BSD license,BSD license


In [6]:
all_packages.to_csv("test_1.csv", index=False)

In [7]:
print(
    f"Number of mismatching linceses {(all_packages.license_new != all_packages.license_old).sum()}"
)

Number of mismatching linceses 179


In [8]:
all_packages[all_packages.license_new != all_packages.license_old].head()

,package_name_old,package_name_new,package_name_general,license_new,license_old
0,aioitertools,aioitertools,aioitertools,MIT license,
0,aioresponses,aioresponses,aioresponses,MIT license,
0,aiosqlite,aiosqlite,aiosqlite,MIT license,
0,alabaster,alabaster,alabaster,BSD license,
0,aniso8601,aniso8601,aniso8601,BSD license,


In [9]:
print(
    f"Number of missing from new packages {len(all_packages[all_packages.package_name_new == ""])}"
)

Number of missing from new packages 0


In [10]:
print(
    f"Number of missing from old packages {len(all_packages[all_packages.package_name_old == ""])}"
)

Number of missing from old packages 16


In [11]:
mismatches = all_packages.loc[
    all_packages.license_new != all_packages.license_old
].sort_values(by=["package_name_general"])

mismatches.to_csv("mismatches.csv", index=False)

mismatches.loc[mismatches.license_old != ""]

,package_name_old,package_name_new,package_name_general,license_new,license_old
0,dbt-extractor,dbt-extractor,dbt_extractor,Apache license,Apache license 2.0
0,deltalake,deltalake,deltalake,Apache license,MIT license
0,hf-xet,hf-xet,hf_xet,Apache license,Apache license 2.0
